The Singleton design pattern is a software engineering principle that ensures a class has only one instance throughout the entire lifecycle of an application. This pattern restricts object instantiation by controlling the creation process, so once the first instance is created, any subsequent calls return a reference to that same instance rather than creating a new one. The Singleton pattern is particularly useful for managing shared resources, coordinating actions, or maintaining global state in a controlled manner.

This design pattern is typically applied in scenarios where multiple instances could cause conflicts, inefficiencies, or inconsistency. Common use cases include:

- Configuration managers: Centralizing configuration settings so all components access a consistent source.

- Logging services: Ensuring a single logging instance that collects and manages logs uniformly.

- Database connection pools: Managing a limited number of database connections to optimize resource usage.

- Thread pools and task schedulers: Controlling the execution of concurrent processes efficiently.

- Caches and registries: Providing a unified access point to frequently used data without duplication.

The fundamental advantage of the Singleton pattern is that it promotes controlled access to a unique instance, ensuring consistency and preventing redundant resource consumption. However, it also introduces challenges, such as potential issues in multithreaded environments if synchronization is not handled properly, and risks becoming an anti-pattern if overused or used as a global variable substitute. Proper implementation and careful consideration of use cases are essential to leverage the Singleton pattern effectively.

Here is a simple ready to use Singleton class:

In [15]:
class Singleton:
    _instance = None
    
    def __init__(self):
        if Singleton._instance is None:
            Singleton._instance = self
        else:
            raise Exception("This is a Singleton class")
    
    @classmethod
    def get_instance(cls):
        if cls._instance is None:
            cls()
        return cls._instance


Now if you run the code below, it shows that the second user cannot be created like this.

In [ ]:
user1 = Singleton()
print(user1)
try:
    user2 = Singleton()
    print(user2)
except Exception as e:
    print('error', e)

This is a Singleton class


what can we do then?
Instead of creating an Instance of the class, we let the class give us an instance instead. we use the classmethod get_instance().

In [3]:
user1 = Singleton.get_instance()
print(user1)

user2 = Singleton.get_instance()
print(user2)

as you can see, it gives the same address each time we make a new instance using this classmethod. now lets make it so that we dont have to use a classmethod, we just call the class object when creating and have it give us the same instance. the code will be simpler, and the user dont have to be aware of the fact that this is a singleton and know that they have to use the classmethod get_instance().

In [11]:
class Singleton:
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

In [ ]:
user1 = Singleton()
print(user1)
user2 = Singleton()
print(user2)

As you can see, you can just directly call the class object and have it be the same instance everytime

there might be a problem when multithreading, as in each thread, it thinks the _instance is None and creates multiple instances. thankfully python has a builtin lock to make it so that it is semi synchronized. meaning at one point only one thread can go to the __new__ function:

In [ ]:
import threading
class Singleton:
    _instance = None
    _lock = threading.Lock()
    
    def __new__(cls):
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    cls._instance = super().__new__(cls)
        return cls._instance
    

Now, why the hell do we check cls._instance is None twice? Because locking is expensive.
1. First check (outside the lock):
    Fast path. If _instance is already set, we avoid the lock entirely. This means most calls after initialization skip locking — boosting performance.

2. Second check (inside the lock):
    Guarantees only one thread creates the instance. Prevents a race condition: imagine two threads reaching the first if at the same time before _instance is set.

    They’d both acquire the lock (one after the other), but without the second check, the second one would overwrite the instance.